## Import Packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


import numpy as np
import pandas as pd
import os 
import re
from matplotlib import cm
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from IPython.display import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from matplotlib.colors import LogNorm, Normalize

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Exploration


In [ ]:
bc_data = r'/content/drive/MyDrive/Land Cover Classification with Sentinel-2/BC_input_data.CSV'
#bc_loc = "content/drive/MyDrive/Land Cover Classification with Sentinel-2/BC_input_data.CSV"
#https://drive.google.com/drive/folders/1oIZgiPBInypzMU1AcwCSlhjLAW1frVpa

In [ ]:
df_bc = pd.read_csv(bc_data)

In [ ]:
#df = pd.read_csv(bc_data)  
print(df_bc.info())
df_bc.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5112785 entries, 0 to 5112784
Data columns (total 13 columns):
 #   Column      Dtype  
---  ------      -----  
 0   B01         float64
 1   B02         float64
 2   B03         float64
 3   B04         float64
 4   B05         float64
 5   B06         float64
 6   B07         float64
 7   B08         float64
 8   B09         float64
 9   B11         float64
 10  B12         float64
 11  B8A         float64
 12  land_cover  int64  
dtypes: float64(12), int64(1)
memory usage: 507.1 MB
None


,B01,B02,B03,B04,B05,B06,B07,B08,B09,B11,B12,B8A,land_cover
0,0.0172,0.0200,0.0373,0.0243,0.0643,0.1630,0.1883,0.1995,0.2100,0.1222,0.0642,0.2045,14
1,0.0161,0.0297,0.0476,0.0447,0.0834,0.1770,0.2067,0.2224,0.2265,0.1829,0.1020,0.2302,14
2,0.0140,0.0151,0.0249,0.0241,0.0459,0.0992,0.1188,0.1263,0.2059,0.1082,0.0585,0.1328,14
3,0.0132,0.0267,0.0435,0.0411,0.0794,0.1701,0.1955,0.2137,0.2261,0.1669,0.0943,0.2197,18
4,0.0142,0.0255,0.0422,0.0389,0.0804,0.1700,0.1958,0.2122,0.2349,0.1546,0.0848,0.2189,18
5,0.0149,0.0261,0.0480,0.0372,0.0959,0.2127,0.2442,0.2639,0.2662,0.1509,0.0806,0.2694,18
6,0.0217,0.0289,0.0481,0.0379,0.0863,0.1801,0.2070,0.2180,0.2344,0.1499,0.0834,0.2239,14
7,0.0278,0.0495,0.0721,0.0628,0.1189,0.2124,0.2368,0.2499,0.2588,0.1794,0.1022,0.2572,14
8,0.0153,0.0172,0.0306,0.0266,0.0602,0.1190,0.1369,0.1463,0.1926,0.0895,0.0486,0.1509,2
9,0.0069,0.0066,0.0119,0.0145,0.0233,0.0281,0.0348,0.0316,0.0949,0.0295,0.0158,0.0360,18


##Create X and y variables for data splitting

In [ ]:
#df_bc = df_bc[0:SAMPLE_SIZE]
y = df_bc.land_cover
X = df_bc.drop(['land_cover'], axis=1)
columns = X.columns

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
print(X.shape)
print(y.shape)

(5112785, 12)
(5112785,)


#Data Splitting into Test and Train

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(4090228, 12) (1022557, 12) (4090228,) (1022557,)


#Create RF Classifier

In [ ]:
clf=RandomForestClassifier(n_estimators=200, max_features='sqrt', min_samples_leaf=2, max_depth=10)

clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [ ]:
# RF with Boosting
#clf=RandomForestClassifier(n_estimators=200, max_features='sqrt', min_samples_leaf=2, max_depth=10)
#Adaboost = AdaBoostClassifier(base_estimator=clf,n_estimators=200,learning_rate=1)
#boostmodel=Adaboost.fit(X_train,y_train)
#y_bpred=boostmodel.predict(X_test)

##Print Accuracy score

In [ ]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
#print("Accuracy:", metrics.accuracy_score(y_test, y_bpred))

Accuracy: 0.6711831223100522


##Checking Feature Importance

In [ ]:
feature_imp = pd.Series(clf.feature_importances_,index=columns).sort_values(ascending=False)
feature_imp

B04    0.228819
B05    0.144974
B02    0.098833
B8A    0.094499
B11    0.086424
B08    0.079986
B07    0.059079
B12    0.055396
B03    0.055069
B06    0.042588
B09    0.032166
B01    0.022166
dtype: float64

## Validation

In [ ]:
on_data = r'/content/drive/MyDrive/Land Cover Classification with Sentinel-2/ON_input_data.CSV'

In [ ]:
df_on = pd.read_csv(on_data)

In [ ]:
y_val = df_on.land_cover
X_val = df_on.drop(['land_cover'], axis=1)
columns = X_val.columns

# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_val = scaler.fit_transform(X_val)

In [ ]:
y_val_pred=clf.predict(X_val)

In [ ]:
print("Accuracy:", metrics.accuracy_score(y_val, y_val_pred))
#print("Accuracy:", metrics.accuracy_score(y_test, y_bpred))

Accuracy: 0.6724280406862405
